In [1]:
import os
from scipy import misc
import numpy as np
from keras.utils.data_utils import get_file
from six.moves import cPickle
import sys
from keras.utils import np_utils 
import cPickle as pickle
from sklearn.cross_validation import train_test_split
import random

Using Theano backend.


In [2]:
#define the working directory
cwd=os.getcwd()
(cwdtop,tail)=os.path.split(cwd)
patch_dir=os.path.join(cwdtop,'patches')
pickle_dir=os.path.join(cwdtop,'pickle')
if not os.path.exists(pickle_dir):
    os.mkdir(pickle_dir)  


In [3]:
print pickle_dir

/Users/peterhirt/datascience/ild-cnn/pickle


In [4]:
# define a list of used labels
usedclassif = ['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']
print (usedclassif)

['consolidation', 'fibrosis', 'ground_glass', 'healthy', 'micronodules', 'reticulation']


In [5]:
# define a dictionary with labels
classif={}
i=0
for f in usedclassif:
    classif[f]=i
    i+=1

In [6]:
print classif

{'micronodules': 4, 'healthy': 3, 'ground_glass': 2, 'consolidation': 0, 'reticulation': 5, 'fibrosis': 1}


In [7]:
# list all directories under patch directory. They are representing the categories

category_list=os.walk( patch_dir).next()[1]


In [8]:
# print what we have as categories
category_list

['consolidation',
 'fibrosis',
 'ground_glass',
 'healthy',
 'micronodules',
 'reticulation']

In [9]:
# creating variables
# list for the merged pixel data
dataset_list = []
# list of the label data
label_list = []



# go through all categories
# 
for category in category_list:
    if category in usedclassif:
        category_dir = os.path.join(patch_dir, category)
        print  'the path into the categories is: ', category_dir

        sub_categories_dir_list = (os.listdir(category_dir))
        print 'the sub categories are : ', sub_categories_dir_list

        for subCategory in sub_categories_dir_list:

            subCategory_dir = os.path.join(category_dir, subCategory)
            print  'the path into the sub categories is: '
            print subCategory_dir

            image_files = (os.listdir(subCategory_dir))

            for file in image_files:

                if file.find('.bmp') > 0:

                    # load the .bmp file into array       
                    image = misc.imread(os.path.join(subCategory_dir,file), flatten= 0)
                    # append the array to the dataset list
                    dataset_list.append(image)

                    # created rotated copies of images
                    image90 = np.rot90(image)
                    dataset_list.append(image90)

                    image180 = np.rot90(image90)
                    dataset_list.append(image180)

                    image270 = np.rot90(image180)
                    dataset_list.append(image270)

                    # append  lables 
                    label_list.append(classif[category])
                    # append also lables for rotated images
                    label_list.append(classif[category])
                    label_list.append(classif[category])
                    label_list.append(classif[category])


            

the path into the categories is:  /Users/peterhirt/datascience/ild-cnn/patches/consolidation
the sub categories are :  ['apical', 'diffuse', 'non-relevant', 'peripheral_sub_pleural']
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/apical
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/diffuse
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/non-relevant
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/consolidation/peripheral_sub_pleural
the path into the categories is:  /Users/peterhirt/datascience/ild-cnn/patches/fibrosis
the sub categories are :  ['apical', 'basal', 'diffuse', 'non-relevant', 'perihilar', 'peripheral_sub_pleural']
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patches/fibrosis/apical
the path into the sub categories is: 
/Users/peterhirt/datascience/ild-cnn/patc

In [10]:
print (len(dataset_list),len(label_list))



(25572, 25572)


In [12]:
nb_elem = 25000 
indices = []  
resultatx=[]
resultaty= [] 
while nb_elem > 0:  
    i = random.randint(0, len(dataset_list) -1)  
    while i in indices: # tant que le tirage redonne un nombre déjà choisi  
        i = random.randint(0, len(dataset_list) -1)  
    indices.append(i)  
    nb_elem = nb_elem - 1  

for index in indices:  
    resultatx.append(dataset_list[index])  
    resultaty.append(label_list[index]) 

# transform dataset list into numpy array                   
dataset = np.array(resultatx)

# this is already in greyscale
# use only one of the 3 color channels as greyscale info
X = dataset[:,:,:]

print 'dataset shape is now: ', X.shape

# 
y = np.array(resultaty)
# sampling item 22
#y[22]


dataset shape is now:  (25000, 32, 32)


In [13]:
print (len(dataset_list),len(label_list))
print (len(resultatx),len(resultaty))

(25572, 25572)
(25000, 25000)


In [14]:
print X.shape
print y.shape

(25000, 32, 32)
(25000,)


In [15]:
X_train, X_intermediate, y_train, y_intermediate = train_test_split(X, y, test_size=0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_intermediate, y_intermediate, test_size=0.5, random_state=42)
print X_train.shape
print X_val.shape
print X_test.shape
print y_train.shape
print y_test.shape
print y_val.shape
    

(12500, 32, 32)
(6250, 32, 32)
(6250, 32, 32)
(12500,)
(6250,)
(6250,)


In [16]:
# save the dataset and label set into serial formatted pkl 

pickle.dump(X_train, open( os.path.join(pickle_dir,"X_train.pkl"), "wb" ))
pickle.dump(X_test, open( os.path.join(pickle_dir,"X_test.pkl"), "wb" ))
pickle.dump(X_val, open(os.path.join(pickle_dir,"X_val.pkl"), "wb" ))
pickle.dump(y_train, open( os.path.join(pickle_dir,"y_train.pkl"), "wb" ))
pickle.dump(y_test, open( os.path.join(pickle_dir,"y_test.pkl"), "wb" ))
pickle.dump(y_val, open( os.path.join(pickle_dir,"y_val.pkl"), "wb" ))

In [17]:
# testing if pickls was working fine
recuperated_X_train = pickle.load( open( os.path.join(pickle_dir,"X_train.pkl"), "rb" ) )

In [18]:
recuperated_X_train

array([[[ 14,  23,  28, ...,  49,  21,   0],
        [ 21,  21,  26, ...,  30,  19,   0],
        [ 16,  16,  37, ...,  37,  58, 119],
        ..., 
        [ 19,  37,  75, ...,  30,  40,  49],
        [ 33,  37,  51, ...,  49,  49,  96],
        [ 30,  33,  40, ...,  73,  91,  91]],

       [[ 71,  86,  95, ..., 154, 178, 190],
        [ 62,  98, 122, ..., 148, 190, 213],
        [ 86,  92,  98, ..., 166, 163, 178],
        ..., 
        [ 92,  83, 113, ...,  44,  50,  62],
        [104, 101,  86, ...,  42,  50,  68],
        [ 92,  95,  56, ...,  68,  59,  59]],

       [[163, 157, 113, ..., 130, 113,  86],
        [102, 121,  98, ..., 134, 121, 115],
        [105, 150, 130, ...,  96, 121, 115],
        ..., 
        [176, 161, 151, ..., 107,  94,  81],
        [159, 171, 167, ...,  81,  63,  71],
        [125, 165, 180, ...,  77,  86,  92]],

       ..., 
       [[ 82, 105,  64, ...,  75, 101, 109],
        [ 86, 109,  94, ..., 131, 135,  90],
        [105, 131, 142, ..., 120, 120, 